# Python converted numerical Hessian

In [16]:
import numpy as np

In [159]:
def quadratic(X):
    return np.sum(X**2)
def cubic(X):
    return (X[0]**3)*(X[1]**2)
def cubic_hes(X):
    return np.array([[6*X[0]*X[1]**2, 6*X[0]**2*X[1]], [6*X[0]**2*X[1], 2*X[0]**3]])

In [160]:
def numHess(func, x):
    if not np.isscalar(func(x)):
        raise ValueError('Richardson method for Hessian assumes a scalar valued function.')
    D = genD(func, x)
    H = np.zeros((len(x), len(x)))
    u = len(x)-1
    for i in range(len(x)):
        for j in range(i+1):
            u+=1
            H[i, j] = D[u]
    H = H + H.T
    np.fill_diagonal(H, np.diag(H)/2)
    return H

In [161]:
def genD(func, x):
    eps = 1e-4
    d = 0.1
    zeroTol = np.sqrt(eps/(7e-7))
    r = 4
    v = 2
    x = x.astype(float)
    f0 = func(x)
    
    n = len(x)
    h0 = np.abs(d*x) + eps * (np.abs(x) < zeroTol)
    D = np.zeros(int(n*(n + 3)/2))
    Dapprox = np.zeros(r)
    Hdiag = np.zeros(n)
    Happrox = np.zeros(r)
    
    for i in range(n):
        h = h0
        for k in range(r):
            x1 = x.copy()
            x1[i] += h[i]
            x2 = x.copy()
            x2[i] -= h[i]
            f1 = func(x1)
            f2 = func(x2)
            Dapprox[k] = (f1 - f2) / (2*h[i])
            Happrox[k] = (f1 - 2*f0 + f2) / h[i]**2
            h = h/v
        for m in range(r-1):
            for k in range(r-m-1):
                Dapprox[k] = (Dapprox[k+1]*(4**(m+1)) - Dapprox[k])/(4**(m+1)-1)
                Happrox[k] = (Happrox[k+1]*(4**(m+1)) - Happrox[k])/(4**(m+1)-1)
        D[i] = Dapprox[0]
        Hdiag[i] = Happrox[0]
    u = n-1
    for i in range(n):
        for j in range(i+1):
            u+=1
            if i==j:
                D[u] = Hdiag[i]
            else:
                h = h0
                for k in range(r):
                    x1 = x.copy()
                    x1[i] += h[i]
                    x1[j] += h[j] 
                    x2 = x.copy()
                    x2[i] -= h[i] 
                    x2[j] -= h[j]
                    f1 = func(x1)
                    f2 = func(x2)
                    Dapprox[k] = (f1 - 2*f0 + f2 - Hdiag[i]*h[i]**2 - Hdiag[j]*h[j]**2)/(2*h[i]*h[j])
                    h = h/v
                for m in range(r-1):
                    for k in range(r-m-1):
                        Dapprox[k] = (Dapprox[k+1]*(4**(m+1)) - Dapprox[k])/(4**(m+1)-1)
                D[u] = Dapprox[0]
    return D

In [ ]:
rosen = lambda x : (1.-x[0])**2 + 105*(x[1]-x[0]**2)**2
numHess(rosen, np.array([1, 1]))